In [1]:
!pip install --quiet transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.0 MB/s eta 0:00:00:00:0100:01


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model     = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")
model.eval()


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

2025-05-24 13:08:50.174116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748092130.474428      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748092130.556197      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# A little persona + seed example to prime the model
PERSONA = (
    "The following is a conversation between a friendly AI assistant and a human.\n"
    "Human: Hello, how are you?\n"
    "Assistant: I'm doing well, thank you! How can I help you today?\n"
)

def chat():
    print("Assistant ready! (type 'exit' to quit)\n")
    history = PERSONA
    while True:
        user = input("You: ").strip()
        if not user or user.lower() == "exit":
            print("Goodbye!")
            break

        # build prompt with persona + conversation so far
        prompt = history + f"Human: {user}\nAssistant:"
        inputs = tokenizer(prompt, return_tensors="pt")

        out = model.generate(
            **inputs,
            max_length=inputs.input_ids.shape[-1] + 50,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_p=0.9,
            top_k=50,
            temperature=0.7
        )

        # extract only the new tokens (the assistant’s reply)
        reply_ids = out[0, inputs.input_ids.shape[-1]:]
        reply = tokenizer.decode(reply_ids, skip_special_tokens=True).strip()

        print(f"Assistant: {reply}\n")

        history += f"Human: {user}\nAssistant: {reply}\n"

chat()


Assistant ready! (type 'exit' to quit)



You:  hello


Assistant: Well I'm doing well, how are you?



You:  hello


Assistant: What is your name? Human : no, I'm not, human :



...

Assistant: 3



You:  ?


Assistant: 3: 3



You:  stoppp


Assistant: 3 : 3 : 3 : 3 : 3 : 3 : 3 : 3 :

